## About
It is anticipated that the `developer` exploring this blueprint will likely follow one of the two paths i.e.
* Build your own conversational agent from the grounds up
* Extend an existing agent

The blueprint has two sets of APIs the application builder is expected to interact with. The blueprint is expected to be deployed using docker compose for this APIs to be accesible.

* Agent apis
This is exposed on port 8081 and accessible on "http://IPADDR:8081".
Api documentation is available at  "http://IPADDR:8081/docs#"

* Analytics server apis
This is exposed on port 8082 and accessible on "http://IPADDR:8082".
Api documentation is available at  "http://IPADDR:8082/docs#" 

This notebook further illustrates one more aspect which becomes important when the `customer service operations` team wants to leverage the user feedback to power the data flywheel. Examples are included on how to glean the feedback data from the blueprint.

In [1]:
!pip install requests

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip


### Notebook variables

In [2]:
IPADDRESS = "localhost" #Replace this with the correct IP address
AGENT_PORT = "8081"
ANALYTICS_PORT = "8082"
AGENT_BASE_URL = f'http://{IPADDRESS}:{AGENT_PORT}'
ANALYTICS_BASE_URL = f'http://{IPADDRESS}:{ANALYTICS_PORT}'

### Agent API usage
The next few set of cells illustrate examples of the APIs as documented at 
http://localhost:8081/docs#

In [28]:
# Health
# Perform a Health Check
import requests
url = AGENT_BASE_URL + "/health"
headers = {
    "accept": "application/json"
}
response = requests.get(url, headers=headers)

# Print the response
print("Status Code:", response.status_code)
print("Response Body:", response.json())

Status Code: 200
Response Body: {'message': 'Service is up.'}


In [29]:
# Metrics

import requests
url = AGENT_BASE_URL + "/metrics"
headers = {
    "accept": "application/json"
}
response = requests.get(url, headers=headers)
# Print the response
print("Status Code:", response.status_code)
try:
    print("Response Body:", response.json())
except ValueError:
    print("Response is not in JSON format:", response.text)

Status Code: 200
Response is not in JSON format: # HELP python_gc_objects_collected_total Objects collected during gc
# TYPE python_gc_objects_collected_total counter
python_gc_objects_collected_total{generation="0"} 11859.0
python_gc_objects_collected_total{generation="1"} 4462.0
python_gc_objects_collected_total{generation="2"} 222.0
# HELP python_gc_objects_uncollectable_total Uncollectable objects found during GC
# TYPE python_gc_objects_uncollectable_total counter
python_gc_objects_uncollectable_total{generation="0"} 0.0
python_gc_objects_uncollectable_total{generation="1"} 0.0
python_gc_objects_uncollectable_total{generation="2"} 0.0
# HELP python_gc_collections_total Number of times this generation was collected
# TYPE python_gc_collections_total counter
python_gc_collections_total{generation="0"} 396.0
python_gc_collections_total{generation="1"} 35.0
python_gc_collections_total{generation="2"} 3.0
# HELP python_info Python platform information
# TYPE python_info gauge
python_in

In [74]:
# create_session
# This needs to be done at the commencement of a conversation.
# The returned the session_id needs to be used in the conversation that ensues
import requests
url = AGENT_BASE_URL + "/create_session"
headers = {
    "accept": "application/json"
}
response = requests.get(url, headers=headers)
# Print the response
print("Status Code:", response.status_code)
if response.status_code == 200:
    try:
        data = response.json()
        session_id = data.get("session_id")
        print("Session ID:", session_id)
    except ValueError:
        print("Response is not in JSON format:", response.text)
else:
    print("Failed to create session. Status Code:", response.status_code)

Status Code: 200
Session ID: d02763c7-e58f-49d7-9e23-ba20899d609c


In [75]:
# generate
# user_id is set to John Doe (refer the customer data csv)
# session_id from the "create_session" is used in the post request

import requests
url = AGENT_BASE_URL + "/generate"  # Replace with the appropriate endpoint
headers = {
    "Content-Type": "application/json",
    "accept": "application/json"
}

# "What is my physics course grade distribution or weighting?"

payload = {
    "messages": [
        {
            "role": "user",
            "content": "What grades have I gotten on my assignments? My student id is 1 "
            # "content": "My user id is 1"
        }
    ],
    "user_id": "1",  # Replace with the actual user ID
    "session_id": f"{session_id}"  # Replace with the actual session ID
}

response = requests.post(url, json=payload, headers=headers)
# Print the response
print("Status Code:", response.status_code)
try:
    print("Response Body:", response.json())
except ValueError:
    print("Response is not in JSON format:", response.text)

Status Code: 200
Response is not in JSON format: data: {"id":"7b943ab1-e5f6-4880-8a7b-8718921cc4f6","choices":[{"index":0,"message":{"role":"assistant","content":"You"},"finish_reason":""}],"session_id":"d02763c7-e58f-49d7-9e23-ba20899d609c"}

data: {"id":"7b943ab1-e5f6-4880-8a7b-8718921cc4f6","choices":[{"index":0,"message":{"role":"assistant","content":" have"},"finish_reason":""}],"session_id":"d02763c7-e58f-49d7-9e23-ba20899d609c"}

data: {"id":"7b943ab1-e5f6-4880-8a7b-8718921cc4f6","choices":[{"index":0,"message":{"role":"assistant","content":" received"},"finish_reason":""}],"session_id":"d02763c7-e58f-49d7-9e23-ba20899d609c"}

data: {"id":"7b943ab1-e5f6-4880-8a7b-8718921cc4f6","choices":[{"index":0,"message":{"role":"assistant","content":" a"},"finish_reason":""}],"session_id":"d02763c7-e58f-49d7-9e23-ba20899d609c"}

data: {"id":"7b943ab1-e5f6-4880-8a7b-8718921cc4f6","choices":[{"index":0,"message":{"role":"assistant","content":" grade"},"finish_reason":""}],"session_id":"d02763

In [51]:
# feedback/response
# The feedback pertains to the most recent response as per "generate" api
# feedback convention: -1:Negative, 0:Neutral, 1=Positive
import requests

url = AGENT_BASE_URL + "/feedback/response"
headers = {
    "accept": "application/json",
    "Content-Type": "application/json"
}
payload = {
    "feedback": -1,
    "session_id": f"{session_id}"  # Replace with the actual session ID
}
response = requests.post(url, json=payload, headers=headers)
# Print the response
print("Status Code:", response.status_code)
try:
    print("Response Body:", response.json())
except ValueError:
    print("Response is not in JSON format:", response.text)

Status Code: 200
Response Body: {'message': 'Response feedback saved successfully'}


In [29]:
# end_session

import requests
url = f"{AGENT_BASE_URL}/end_session?session_id={session_id}"
headers = {
    "accept": "application/json"
}
response = requests.get(url, headers=headers)
# Print the response
print("Status Code:", response.status_code)
try:
    print("Response Body:", response.json())
except ValueError:
    print("Response is not in JSON format:", response.text)

Status Code: 200
Response Body: {'message': 'Session ended'}


### Analytics API usage
The next few set of cells illustrate examples of the APIs as documented at 
http://localhost:8082/docs#

In [27]:
# Health
# performs a health check
import requests
url = ANALYTICS_BASE_URL + "/health"
headers = {
    "accept": "application/json"
}
response = requests.get(url, headers=headers)

# Print the response
print("Status Code:", response.status_code)
print("Response Body:", response.json())

Status Code: 200
Response Body: {'message': 'Service is up.'}


In [66]:
# sessions
# Retrieve session information in last 2 hours

import requests

url = f"{ANALYTICS_BASE_URL}/sessions?hours=2"
headers = {
    "accept": "application/json"
}
response = requests.get(url, headers=headers)
# Print the response
print("Status Code:", response.status_code)
try:
    print("Response Body:", response.json())
except ValueError:
    print("Response is not in JSON format:", response.text)

Status Code: 200
Response Body: []


In [67]:
# conversation?session_id=xyz
# fetch the conversation history given a session id

import requests
print("session_id :{}".format(session_id))
url = f"{ANALYTICS_BASE_URL}/session/conversation?session_id={session_id}"
headers = {
    "accept": "application/json"
}
response = requests.get(url, headers=headers)
# Print the response
print("Status Code:", response.status_code)
try:
    print("Response Body:", response.json())
except ValueError:
    print("Response is not in JSON format:", response.text)

session_id :ba7adc61-5486-4e2e-b928-1b930aa5dccf
Status Code: 404
Response Body: {'detail': 'Session not found. Please check the session ID or end the session.'}


In [68]:
# session/summary
# generate the conversation summary given a session_id

import requests
print("session_id :{}".format(session_id))
url = f"{ANALYTICS_BASE_URL}/session/summary?session_id={session_id}"
headers = {
    "accept": "application/json"
}
response = requests.get(url, headers=headers)
# Print the response
print("Status Code:", response.status_code)
try:
    print("Response Body:", response.json())
except ValueError:
    print("Response is not in JSON format:", response.text)

session_id :ba7adc61-5486-4e2e-b928-1b930aa5dccf
Status Code: 200
Response Body: {'session_info': {'session_id': 'ba7adc61-5486-4e2e-b928-1b930aa5dccf', 'start_time': None, 'end_time': None}, 'summary': 'Something went wrong. Could you try again in a few seconds with a different conversation.', 'sentiment': 'neutral'}


In [27]:
## feedback/summary
## store the feedback for the summary generated by the solution
# feedback convention: -1:Negative, 0:Neutral, 1=Positive

url = ANALYTICS_BASE_URL + "/feedback/summary"
headers = {
    "accept": "application/json",
    "Content-Type": "application/json"
}
payload = {
    "feedback": +1, # positive
    "session_id": f"{session_id}"  # Replace with the actual session ID
}
response = requests.post(url, json=payload, headers=headers)
# Print the response
print("Status Code:", response.status_code)
try:
    print("Response Body:", response.json())
except ValueError:
    print("Response is not in JSON format:", response.text)

Status Code: 200
Response Body: {'message': 'Summary feedback saved successfully'}


In [28]:
## feedback/session
## store user feedback for the overall conversation session.
# feedback convention: -1:Negative, 0:Neutral, 1=Positive

url = ANALYTICS_BASE_URL + "/feedback/session"
headers = {
    "accept": "application/json",
    "Content-Type": "application/json"
}
payload = {
    "feedback": -1, # negative
    "session_id": f"{session_id}"  # Replace with the actual session ID
}
response = requests.post(url, json=payload, headers=headers)
# Print the response
print("Status Code:", response.status_code)
try:
    print("Response Body:", response.json())
except ValueError:
    print("Response is not in JSON format:", response.text)

Status Code: 200
Response Body: {'message': 'Session feedback saved successfully'}


In [29]:
## feedback/sentiment
## store rating for the sentiment generated by the solution
# feedback convention: -1:Negative, 0:Neutral, 1=Positive

url = ANALYTICS_BASE_URL + "/feedback/sentiment"
headers = {
    "accept": "application/json",
    "Content-Type": "application/json"
}
payload = {
    "feedback": 0, # neutral
    "session_id": f"{session_id}"  # Replace with the actual session ID
}
response = requests.post(url, json=payload, headers=headers)
# Print the response
print("Status Code:", response.status_code)
try:
    print("Response Body:", response.json())
except ValueError:
    print("Response is not in JSON format:", response.text)

Status Code: 200
Response Body: {'message': 'Sentiment feedback saved successfully'}


In [30]:
# delete_session

import requests
url = f"{AGENT_BASE_URL}/delete_session?session_id={session_id}"
headers = {
    "accept": "application/json"
}
response = requests.delete(url, headers=headers)
# Print the response
print("Status Code:", response.status_code)
try:
    print("Response Body:", response.json())
except ValueError:
    print("Response is not in JSON format:", response.text)

Status Code: 200
Response Body: {'message': 'Session info deleted'}


### Accessing User Feedback data
The next few set of cells illustrate how the various types of feedback data can be collected to power the data flywheel.
Refer to the docker-compose.yaml or helm chart for the credentials of the postgres db

In [31]:
POSTGRES_HOST = "localhost"
POSTGRESDB_PORT = "5432"
POSTGRES_USER = "postgres"
POSTGRES_PASSWD = "password"
POSTGRES_DBNAME = "postgres"
FEEDBACK_TBLNAME = "feedback"

#### Schema information of the `feedback` table

In [36]:
!pip install psycopg2

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 385.7/385.7 kB 69.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [23 lines of output]
      running egg_info
      creating /tmp/pip-pip-egg-info-25up93cg/psycopg2.egg-info
      writing /tmp/pip-pip-egg-info-25up93cg/psycopg2.egg-info/PKG-INFO
      writing dependency_links to /tmp/pip-pip-egg-info-25up93cg/psycopg2.egg-info/dependency_links.txt
      writing top-level names to /tmp/pip-pip-egg-info-25up93cg/psycopg2.egg-info/top_level.txt
      writing manifest file '/tmp/pip-pip-egg-info-25up93cg/psycopg2.egg-info/SOURCES.txt'
      
      Error: pg_config executable not found.
      
      pg_config is required to build psycopg2 from source.  Please add the directory
      containing pg_config to the $PATH or specify the full executable

In [32]:
import psycopg2

# Connection details
host = POSTGRES_HOST
port = POSTGRESDB_PORT
database = POSTGRES_DBNAME
user = POSTGRES_USER
password = POSTGRES_PASSWD

# The schema and table you're interested in
schema_name = "public"      # replace if needed
table_name = FEEDBACK_TBLNAME   # replace with the actual table name

try:
    # Connect to the PostgreSQL database
    conn = psycopg2.connect(
        host=host,
        port=port,
        database=database,
        user=user,
        password=password
    )
    cursor = conn.cursor()

    # Query to get column details of a specific table
    # information_schema.columns provides column_name and data_type
    query = """
        SELECT column_name, data_type
        FROM information_schema.columns
        WHERE table_name = %s AND table_schema = %s
        ORDER BY ordinal_position;
    """
    cursor.execute(query, (table_name, schema_name))

    columns = cursor.fetchall()

    # Print the schema details
    print(f"Schema for {schema_name}.{table_name}:")
    for col in columns:
        col_name, data_type = col
        print(f" - {col_name}: {data_type}")

except Exception as e:
    print("Error:", e)
finally:
    if 'cursor' in locals():
        cursor.close()
    if 'conn' in locals():
        conn.close()

ModuleNotFoundError: No module named 'psycopg2'

#### Retrieve the feedback information for each session(session_id)
* sentiment
* summary
* session

These fields can take on a value such as

1:Positive,
0:Neutral,
-1:Negative

In [ ]:
import psycopg2

# Database connection parameters
db_params = {
    'dbname': POSTGRES_DBNAME,
    'user': POSTGRES_USER,
    'password': POSTGRES_PASSWD,
    'host': POSTGRES_HOST,      # e.g., 'localhost' or the IP address
    'port': POSTGRESDB_PORT   # e.g., '5432'
}

# Connect to the database
conn = psycopg2.connect(**db_params)
cur = conn.cursor()

# Query to select the first 5 rows from the customer_data table
query = f'SELECT session_id, sentiment, summary, session FROM feedback;'
# Execute the query
cur.execute(query)
rows = cur.fetchall()

# Print the headers and the corresponding rows
for i, row in enumerate(rows, start=1):
    print(f"{i}:{row}")

    # Close the connection
cur.close()
conn.close()

1:('string', -1.0, None, None)
2:('188b3c82-3384-4c6b-94cb-024d24b312d2', -1.0, 1.0, None)
